In [4]:
import pandas as pd
import sklearn 
import scipy
from sklearn import linear_model as lm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_validation import KFold, train_test_split, cross_val_score, StratifiedKFold, LabelKFold, ShuffleSplit
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from mhcflurry.amino_acid import common_amino_acids
import matplotlib.pyplot as plt 
% matplotlib inline
import numpy as np
import math 
from mhcflurry import peptide_encoding, amino_acid
from mhcflurry.amino_acid import common_amino_acids
import statsmodels.api as sm
from keras import models, layers, optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge, Convolution1D, AveragePooling1D, Activation, Flatten
from keras.preprocessing import sequence
from keras.models import Model
from keras.engine import topology
import seaborn as sns

# Data preparation

In [6]:
df = pd.read_table("bdata.2009.mhci.public.1.txt")

df['log_meas']=1-np.log(df['meas'])/math.log(50000)
df['peptide_length'] = df['sequence'].str.len()


max_len=df['sequence'].str.len().max()
n_peptides = df['sequence'].count()

def amino_acid_hotshot_encoding(s):
    return common_amino_acids.hotshot_encoding([s],len(s)).flatten().astype(int)
df['hotshot_encoded_peptides'] = df.sequence.apply(lambda seq: amino_acid_hotshot_encoding(seq))

def amino_acid_index_encoding(s, maxlen):
    a = 1+common_amino_acids.index_encoding([s],len(s)).flatten()
    return np.concatenate([a, np.zeros(maxlen-len(a),dtype=int)])
df['index_encoded_peptides'] = df.sequence.apply(lambda seq: amino_acid_index_encoding(seq, max_len))

def measured_affinity_less_than(Y,k):
    IC50 = 50000**(1-Y)
    return (IC50 < k).astype(int) 

def affinity_label(Y):
    return measured_affinity_less_than(Y,50) + measured_affinity_less_than(Y,500) + measured_affinity_less_than(Y,5000) + measured_affinity_less_than(Y,50000)

df['affinity_label'] = affinity_label(df['log_meas'])
df = df.reindex(np.random.permutation(df.index))
df.head(10)

,species,mhc,peptide_length,cv,sequence,inequality,meas,log_meas,hotshot_encoded_peptides,index_encoded_peptides,affinity_label
19307,human,HLA-A-0201,9,TBD,RMYGVLPWI,=,29.000000,0.688783,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[15, 11, 20, 6, 18, 10, 13, 19, 8, 0, 0, 0, 0,...",4
20859,human,HLA-A-0201,9,TBD,WLWYIKIFI,=,141.000000,0.542619,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[19, 10, 19, 20, 8, 9, 8, 5, 8, 0, 0, 0, 0, 0,...",3
47641,human,HLA-A-1101,10,TBD,TSFYLISIFL,>,69767.441860,-0.030790,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[17, 16, 5, 20, 10, 8, 16, 8, 5, 10, 0, 0, 0, ...",0
82048,human,HLA-A-6802,9,TBD,LLMALPHQA,=,9880.994830,0.149856,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[10, 10, 11, 1, 10, 13, 7, 14, 1, 0, 0, 0, 0, ...",1
72473,human,HLA-A-3201,9,TBD,RTAFGGKYM,=,4959.546430,0.213563,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[15, 17, 1, 5, 6, 6, 9, 20, 11, 0, 0, 0, 0, 0,...",2
40899,human,HLA-A-0301,10,TBD,LQDPRVRGLY,=,5547.767863,0.203204,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[10, 14, 3, 13, 15, 18, 15, 6, 10, 20, 0, 0, 0...",1
129055,macaque,Mamu-A-11,9,TBD,QDNQWSYKI,=,135.622923,0.546213,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[14, 3, 12, 14, 19, 16, 20, 9, 8, 0, 0, 0, 0, ...",3
81048,human,HLA-A-6802,9,TBD,FLITGVFDI,=,61.996851,0.618561,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 10, 8, 17, 6, 18, 5, 3, 8, 0, 0, 0, 0, 0, ...",3
130084,macaque,Mamu-B-01,11,TBD,RQAPGKGLEWV,>,64285.714286,-0.023227,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[15, 14, 1, 13, 6, 9, 6, 10, 4, 19, 18, 0, 0, ...",0
111327,human,HLA-B-4402,9,TBD,GHMMVIFRL,>,20000.000000,0.084687,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 7, 11, 11, 18, 8, 5, 15, 10, 0, 0, 0, 0, 0...",1


In [7]:
df.groupby('mhc').size().nlargest(11)

mhc
HLA-A-0201    9565
HLA-A-0301    6141
HLA-A-0203    5542
HLA-A-1101    5399
HLA-A-0206    4827
HLA-A-3101    4796
HLA-A-6802    4768
HLA-A-0202    3919
HLA-A-0101    3725
HLA-B-0702    3412
H-2-Kb        3407
dtype: int64

## Only 9 mers

In [8]:
df_9 = df[df['peptide_length']==9]
df_9.groupby('mhc').size().nlargest(11)

mhc
HLA-A-0201    6961
HLA-A-0301    4601
HLA-A-0203    3937
HLA-A-1101    3862
HLA-A-3101    3309
HLA-A-0206    3223
HLA-A-6802    3188
HLA-A-0101    3169
HLA-B-1501    3142
HLA-B-0702    2974
HLA-B-5801    2444
dtype: int64

## HLA-A-0201

In [10]:
df_h = df_9[df_9['mhc']=='HLA-A-0201'][['hotshot_encoded_peptides','index_encoded_peptides','log_meas']]

In [18]:
len(X)

6961

In [29]:
batch_size = 16
epoch = 0
n_epochs = 30 
X = np.array(list(df_h['hotshot_encoded_peptides']))
X = X.reshape((len(X),9,20))
y = np.array(list(df_h['log_meas']))
labels = affinity_label(y)
best_score = 0
for hidden in [50,60]:
    for dropout_probability in [0.25]:
        peak_score = []
        for train_idx, test_idx in StratifiedKFold(labels, n_folds = 5):
            train_aucs = []
            test_aucs = []

            sequence = Input( shape= (9, 20),dtype='float32')
            #embedded = Embedding(21, 100)(sequence)
            forwards = LSTM(hidden)(sequence)
            backwards = LSTM(hidden, go_backwards=True)(sequence)

            merged = merge([forwards, backwards], mode = 'concat', concat_axis=-1)
            after_dp = Dropout(dropout_probability)(merged)
            output = Dense(1, activation = 'sigmoid')(after_dp)
            model = Model(input = sequence, output = output)


            #Compile the model
            model.compile(optimizer = 'adam', loss='mean_squared_error')
            for epoch in range(n_epochs):
                for batch_idx in range(len(X[train_idx]) // batch_size):   
                    model.train_on_batch(X[train_idx][batch_idx * batch_size:(batch_idx + 1) * batch_size], y[train_idx][batch_idx * batch_size:(batch_idx + 1) * batch_size])
                train_pred = model.predict(X[train_idx])
                test_pred = model.predict(X[test_idx])
                train_aucs.append(roc_auc_score(measured_affinity_less_than(y[train_idx],500),train_pred))
                test_aucs.append(roc_auc_score(measured_affinity_less_than(y[test_idx],500),test_pred))
            peak_score.append(max(test_aucs))
    
            print(roc_auc_score(measured_affinity_less_than(y[train_idx],500),train_pred), roc_auc_score(measured_affinity_less_than(y[test_idx],500),test_pred), max(test_aucs), pd.Series(test_aucs).argmax())
        score = np.array(peak_score).mean()
        if score > best_score:
            best_score = score
            best_parameters = {'hidden': hidden ,'dropout':dropout_probability}
print("test score with best parameters using cv is: %.3f" %best_score)
print("with parameters:" , best_parameters)            

0.984547507178 0.959550101958 0.965096496801 16
0.985918550659 0.952630449299 0.961819135744 14
0.986322316762 0.946593296409 0.952511459129 17
0.986175162295 0.942033517953 0.950787815126 11
0.98255003048 0.944671596138 0.953507757091 11
0.987073233216 0.952704551565 0.961629702598 10
0.987128324341 0.954891252504 0.9608580559 18
0.984945643094 0.940944900688 0.954392666157 12
0.989472899115 0.942129010695 0.949742169595 9
0.985743306789 0.942041586724 0.951831066208 13
test score with best parameters using cv is: 0.957
with parameters: {'hidden': 50, 'dropout': 0.25}
